# Testing GEE task api with service user
We want to start tasks and check with the service user whether we can run EE tasks.

In [1]:
import ee
import pathlib

from google.oauth2.service_account import Credentials

In [2]:
ee_credentials: Credentials = ee.ServiceAccountCredentials(
    '578920177147-ul189ho0h6f559k074lrodsd7i7b84rc@developer.gserviceaccount.com',
    str(pathlib.Path.cwd() / "privatekey.json")
)
ee.Initialize(ee_credentials)

Start a task and check whether you get the task back in the task list

In [14]:
# Based on: https://github.com/google/earthengine-api/blob/d60fdfcc5fe6a508db704faeba448f98737210cc/python/ee/tests/batch_test.py#L78
region = [0, 0, 1, 0, 1, 1]
task = ee.batch.Export.image.toCloudStorage(
          ee.Image(42), assetId='users/foo/bar', region=region, scale=1000)
task.start()  # fails as "Asset 'projects/earthengine-legacy/assets/users/foo' does not exist or doesn't allow this operation."

In [25]:
# Based on: https://github.com/google/earthengine-api/blob/d60fdfcc5fe6a508db704faeba448f98737210cc/python/ee/tests/batch_test.py#L306
test_bucket = 'rws-bathymetry-test'
region = ee.Geometry.Rectangle(1, 2, 3, 4)
config = dict(
    region=region['coordinates'],
    maxPixels=10**10,
    outputBucket=test_bucket)
task = ee.batch.Export.image.toCloudStorage(
    ee.Image(1), 'TestDescription', config['outputBucket'], None, None,
    config['region'], None, None, None, config['maxPixels'], None,
    [512, 2048], True)
task.start()

In [3]:
statuses = ee.data.getTaskList()
print(statuses)

[{'state': 'COMPLETED', 'description': 'TestDescription', 'creation_timestamp_ms': 1631620972120, 'update_timestamp_ms': 1631621030694, 'start_timestamp_ms': 1631621007852, 'task_type': 'EXPORT_IMAGE', 'destination_uris': ['https://console.developers.google.com/storage/browser/rws-bathymetry-test/'], 'attempt': 1, 'id': 'JGTSBPNNYQTNYSZDFIIX4J7I', 'name': 'projects/earthengine-legacy/operations/JGTSBPNNYQTNYSZDFIIX4J7I'}, {'state': 'FAILED', 'description': 'myExportImageTask', 'creation_timestamp_ms': 1631613168613, 'update_timestamp_ms': 1631613207864, 'start_timestamp_ms': 1631613207330, 'task_type': 'EXPORT_IMAGE', 'attempt': 1, 'error_message': "Asset 'projects/earthengine-legacy/assets/users/foo' does not exist or doesn't allow this operation.", 'id': 'MKL53WUFDLVMLEQFXRJMMTXD', 'name': 'projects/earthengine-legacy/operations/MKL53WUFDLVMLEQFXRJMMTXD'}]
